In [ ]:
%load_ext autoreload
%autoreload 2

### Handle imports.

In [ ]:
# Allow parent-directory imports.
import sys
sys.path.insert(0, '..')

from pathlib import Path
from utils.analysis import process_sheets, process_makeup_sheets, process_essays, process_essay_makeup_sheets
from utils.helpers import count_completes, determine_pass
from utils.loaders import *
from utils.constants import *

# Increase pandas rows to display more rows.
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 1000)

In [ ]:
# Load roster. Drop/add students as needed.
df = get_roster()

In [ ]:
# Load sheets.
attendance_sheets = get_attendances()
survey_sheets = get_surveys()
makeup_sheets = get_makeups()
essay_sheets = get_essays()
makeup_essay_sheets = get_makeup_essays()

### Process sheets.

In [ ]:
# Everyone gets attendance and survey points for the first lecture.
df['Attendance (08/25) Status'] = 'Complete'

df = process_sheets(attendance_sheets, form_name='Attendance', reference_time=(ATTEND_HR, ATTEND_MN), df=df)

In [ ]:
# Everyone gets attendance and survey points for the first lecture.
df['Survey (08/25) Status'] = 'Complete'
   
df = process_sheets(survey_sheets, form_name='Survey', reference_time=(SURVEY_HR, SURVEY_MN), df=df)

### Process make-up assignments for attendance and surveys.

In [ ]:
# Quick fix for those who answered the make-up assignment to have it also appear for the attendance for the two early lectures.
condition_1 = (df['Survey (08/31) Status'] == 'Complete (Make-up Question Answered)') & (df['Attendance (09/01) Status'] == '*Missing*')
df.loc[condition_1, 'Attendance (09/01) Status'] = 'Complete (Make-up Question Answered)'

condition_2 = (df['Survey (09/06) Status'] == 'Complete (Make-up Question Answered)') & (df['Attendance (09/08) Status'] == '*Missing*')
df.loc[condition_2, 'Attendance (09/08) Status'] = 'Complete (Make-up Question Answered)'

In [ ]:
df = process_makeup_sheets(makeup_sheets, df)

### Process essays.

In [ ]:
df = process_essays(essay_sheets, df)

### Process makeup essays.

In [ ]:
df = process_essay_makeup_sheets(makeup_essay_sheets, df)

### Manual adjustments.

In [ ]:
# As necessary...

### Add "Completeness" columns.

In [ ]:
df['Attendance Count'] = df.apply(lambda x: count_completes(x[ATTENDANCE_COLS]), axis=1)
df['Attendance'] = df['Attendance Count'].apply(lambda x: determine_pass(x, 'Attendance'))

df['Survey Count'] = df.apply(lambda x: count_completes(x[SURVEY_COLS]), axis=1)
df['Survey'] = df['Survey Count'].apply(lambda x: determine_pass(x, 'Survey'))

df['All Requirements Count'] =  df.apply(lambda x: count_completes(x[CLASS_REQS]), axis=1)
df['All Requirements'] = df['All Requirements Count'].apply(lambda x: determine_pass(x, 'Class'))

### Observe the results.

In [ ]:
# Any queries can be run here.
df

### Save results. (change as needed)

In [ ]:
# Save the master spreadsheet.
output_f = os.path.join('..', 'output', 'submissions.xlsx')
df.to_excel(output_f, index=False)

In [ ]:
# Only include complete/incomplete columns.
df = df[['Student ID'] + ATTENDANCE_COLS + SURVEY_COLS + ESSAY_COLS]

# Save individual files.
full_output_dir = os.path.join('output', 'Individual Records')
for idx, row in df.iterrows():
    student_s = row.copy()
    SID = student_s['Student ID'] 
    
    # Create folder for the individual student.
    student_dir = os.path.join(full_output_dir, '{}_CS195'.format(SID))
    Path(student_dir).mkdir(parents=True, exist_ok=True)

    # Write the Series to csv.
    output_f = os.path.join(student_dir, 'records_{}.csv'.format('12_07'))
    student_s.to_csv(output_f, header=False)